In [ ]:
import os
import numpy as np
import pandas as pd
import os
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
audio_dataset_path='C:/Users/ACE/Downloads/Data/genres_original'
metadata=pd.read_csv('C:/Users/ACE/Downloads/Data/features_30_sec.csv')
metadata.head()

In [ ]:
#Feature Extractor
def features_extractor(file):
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    
    return mfccs_scaled_features    


In [ ]:
#metadata.drop(labels=552, axis=0, inplace=True)

In [ ]:
from tqdm import tqdm
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    try:
        final_class_labels=row["label"]
        file_name=os.path.join(os.path.abspath(audio_dataset_path), final_class_labels+'/',str(row["filename"]))
        data=features_extractor(file_name)
        extracted_features.append([data,final_class_labels])
    except Exception as e:
        print(f"Error: {e}")
        continue


In [ ]:
extracted_features_df=pd.DataFrame(extracted_features, columns=['feature', 'class'])
extracted_features_df.head();

In [ ]:
extracted_features_df['class'].value_counts()

In [ ]:
x=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
x.shape

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
x_train

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
num_labels=y.shape[1]

In [ ]:
model=Sequential()
model.add(Dense(1024,input_shape=(40,), activation="relu"))
model.add(Dropout(0,3))
model.add(Dense(512,activation="relu"))
model.add(Dropout(0,3))
model.add(Dense(256,activation="relu"))
model.add(Dropout(0,3))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0,3))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0,3))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0,3))

model.add(Dense(num_labels, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
import time
t=time.localtime()
current_time=time.strftime("%H:%M:%S", t)

In [ ]:
import tensorflow
import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs=100
num_batch_size=32

checkpointer = keras.models.load_model(
    filepath=f'save_models/audio_classification_{current_time}.h5',
    verbose=1,
    save_best_only=True
)

start=datetime.now()

history=model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)

duration=datetime.now()-start

print("Training completed in time :", duration)

In [ ]:
model.evaluate(x_test,y_test, verbose=0)

In [ ]:
#pd.DataFrame(history.history).plot(figsize=(12,6))
#plt.show()

In [ ]:
model.predict_classes(x_test)

In [ ]:
filename="C:/Users/ACE/Downloads/Data/genres_original/rock/rock.00099.wav"
audio,sample_rate=librosa.load(filename,res_type='kaiser_fast')
mfccs_features=librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
mfccs_scaled_features=np.mean(mfccs_features.T,axis=0)

print (mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print (mfccs_scaled_features)
print (mfccs_scaled_features.shape)
predicted_label=model.predict_classes(mfccs_scaled_features)
print (predicted_label)
prediction_class = LabelEncoder.inverse_transform(predicted_label)
prediction_class